In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx


In order to maintain a good CRM (Customer Relationship Management), a company should be aware of each customer’s attitude towards the company. The company should know the answers to the following questions:     
1. when did the customer last purchased? (Recency)   
2. how often does the customer purchase?  (Frequency)  
3. how much money did the customer spend?  (Monetary)   

The answers to the above questions would make the customer be categorized and after this process, the company would be dealing with tens of segments instead of tens of thousands customers. **RFM** is an acronym that stands for **Recency**, **Frequency** and **Monetary**. In order to assign each customer into the appropriate segment, RFM metrics should be calculated and afterwards RFM scores should be computed.


## Getting to know the variables:

**InvoiceNo**   : The number of the invoice, unique per each purchase. Refund invoice numbers contain "C"  
**StockCode**   : Unique code per each item  
**Description** : Name of the item  
**Quantity**    : The number of items within the invoice  
**InvoiceDate** : Date and time of the purchase  
**UnitPrice**   : Price of a single item, as of Sterlin  
**CustomerID**  : Unique id number per each customer  
**Country**     : The country where the customer is living  


## install packages and set configurations


In [2]:
import numpy as np
import pandas as pd
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Read data

In [3]:
df_ = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", 
                    sheet_name="Year 2009-2010") 
df = df_.copy()
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


## Data Understanding

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


how many countries in df:

In [5]:

df['Country'].nunique()

40

the names of the countries with the total values:

In [6]:
df['Country'].value_counts()

United Kingdom          485852
EIRE                      9670
Germany                   8129
France                    5772
Netherlands               2769
Spain                     1278
Switzerland               1187
Portugal                  1101
Belgium                   1054
Channel Islands            906
Sweden                     902
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
United Arab Emirates       432
Denmark                    428
Norway                     369
Finland                    354
Unspecified                310
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
RSA                        111
Bahrain                    107
Canada                      77
Hong Kong                   76
Thailand                    76
Israel  

the most expensive products:

In [7]:
df.sort_values(by='Price', ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09000,17399.00000,United Kingdom
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09000,nan,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,-1,2010-08-27 13:42:00,18910.69000,nan,United Kingdom
517953,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,13541.33000,nan,United Kingdom
519294,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541.33000,nan,United Kingdom


number of unique products:

In [8]:
df['Description'].nunique()

4681

most purchased items:

In [9]:
df.groupby("Description").agg({"Quantity": lambda x: x.sum()}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


check the number of uniques for StockCode ve Description variables:

In [10]:
print(f"Number of uniques in StockCode: {df['StockCode'].nunique()}")
print(f"Number of uniques in Description:{df['Description'].nunique()}")

Number of uniques in StockCode: 4632
Number of uniques in Description:4681


the values were expected to be equal, so there must be more than one unique value in Description variable for one unique StockCode. let's check each StockCode value with the corresponding Description values, get every StockCode that has more than one unique Description in a list form (I hid the output due to its number of rows):

In [11]:
a =df.groupby('StockCode').agg({'Description': "nunique"})
a.reset_index(inplace=True)
a.head()
b = list(a.loc[a['Description'] > 1, 'StockCode'])   

for urun_kodu in b:
    print(f"urun kodu = {urun_kodu} {df.loc[df['StockCode'] == urun_kodu, 'Description'].unique()}")

urun kodu = 10120 ['DOGGY RUBBER' 'Zebra invcing error']
urun kodu = 16011 [' ANIMAL STICKERS' 'ANIMAL STICKERS']
urun kodu = 16012 ['FOOD/DRINK SPUNGE STICKERS' 'FOOD/DRINK SPONGE STICKERS']
urun kodu = 16235 ['RECYCLED PENCIL WITH RABBIT ERASER' '?']
urun kodu = 17033 ['BROCADE PURSE,SMALL ARCH BUTTON' 'sold as 17003?' 'Sold as 17003?' nan]
urun kodu = 20615 ['BLUE SPOTTY PASSPORT COVER' 'BLUE POLKADOT PASSPORT COVER']
urun kodu = 20652 ['BLUE SPOTTY LUGGAGE TAG ' nan 'BLUE POLKADOT LUGGAGE TAG ']
urun kodu = 20658 ['RED SPOTTY LUGGAGE TAG' 'RED RETROSPOT LUGGAGE TAG']
urun kodu = 20661 ['BLUE SPOTTY PURSE ' 'BLUE POLKADOT PURSE ']
urun kodu = 20665 ['RED SPOTTY PURSE ' 'RED RETROSPOT PURSE ']
urun kodu = 20674 ['GREEN SPOTTY BOWL' 'GREEN POLKADOT BOWL']
urun kodu = 20675 ['BLUE SPOTTY BOWL' 'BLUE POLKADOT BOWL']
urun kodu = 20676 ['RED SPOTTY BOWL' 'RED RETROSPOT BOWL']
urun kodu = 20677 ['PINK SPOTTY BOWL' 'PINK POLKADOT BOWL']
urun kodu = 20679 ['EDWARDIAN PARASOL RED' nan '?']
ur

as an example derived from the above list, both the Descriptions: 'PINK SPOTTY BOWL' and 'PINK POLKADOT BOWL' have the same StockCode: 20677. This means that there are duplicates in Description variable (possibly due to manuel entries or merge) so it would be better to use StockCode.

In [12]:
df.loc[df['StockCode'] == 20677, 'Description'].unique() 

array(['PINK SPOTTY BOWL', 'PINK POLKADOT BOWL'], dtype=object)

## Data Preperation

drop na values

In [13]:
df.dropna(inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [15]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,417534.00000,12.75881,101.22042,-9360.00000,-2.00000,1.00000,1.00000,2.00000,4.00000,12.00000,24.00000,36.00000,144.00000,19152.00000
Price,417534.00000,3.88755,71.13180,0.00000,0.29000,0.42000,0.65000,1.25000,1.95000,3.75000,6.75000,8.50000,14.95000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,12435.00000,12725.00000,13042.00000,13983.00000,15311.00000,16799.00000,17706.00000,17913.00000,18196.00000,18287.00000


there are negative values on Quantity variable, this is caused by the refund invoices (Invoices containing the letter "C"), reassign df without refund invoices

In [16]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [17]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,407695.00000,13.58669,96.84223,1.00000,1.00000,1.00000,1.00000,2.00000,5.00000,12.00000,24.00000,36.00000,144.00000,19152.00000
Price,407695.00000,3.29419,34.75666,0.00000,0.29000,0.42000,0.65000,1.25000,1.95000,3.75000,6.75000,8.50000,14.95000,10953.50000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,12435.00000,12731.00000,13044.00000,13997.00000,15321.00000,16812.00000,17706.00000,17913.00000,18196.00000,18287.00000


negative values are excluded. We are not removing outliers (such as the max value on Quantity and Price variables) because we will be scoring the dataset.

In [18]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [19]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


## RFM Metrics

the last date of purchase:

In [20]:
df['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

assign "today's date" as 2 days after the last date of purchase to make sure that none of the Recency values become zero

In [21]:
today_date = df['InvoiceDate'].max() + dt.timedelta(days=2)
today_date

Timestamp('2010-12-11 20:01:00')

create a new df called rfm in order to calculate Recency, Frequency and Monetary values.  
df is grouped by customers and:   

* the number of days between today_date and the last purchase date of this customer is **Recency**  
* the number of unique invoices of this customer is **Frequency**  
* the sum of TotalPrice is this customer's **Monetary**  


In [22]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                    'Invoice': lambda inv: inv.nunique(),
                                    'TotalPrice': lambda price: price.sum()})

In [23]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


renaming rfm columns:

In [24]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


check if there are any zeros in rfm:

In [25]:
rfm.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Recency,4314.00000,92.26912,96.94348,2.00000,2.00000,4.00000,9.00000,19.00000,54.00000,137.00000,256.00000,305.00000,369.00000,375.00000
Frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,1.00000,1.00000,1.00000,2.00000,5.00000,9.00000,13.00000,31.00000,205.00000
Monetary,4314.00000,2047.28866,8912.52324,0.00000,39.95650,110.43650,155.48500,307.95000,705.55000,1722.80250,3796.59500,6235.12050,20136.78250,349164.35000


## RFM Scores

* the min number of Recency metric means that this customer has just purchased, so the highest score (5) should be given to the lower number of Recency.  
* the max number of Frequency and Monetary metrics mean that the customer is purchasing frequently and spending more money, so the highest score (5) should be given to the highest Frequency and Monetary values.

In [26]:
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [27]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252
12347.00000,4,2,1323.32000,5,2,4,524
12348.00000,75,1,222.16000,2,1,1,211
12349.00000,44,3,2671.14000,3,3,5,335
12351.00000,12,1,300.93000,5,1,2,512


display some of the customers with the highest scores:

In [28]:
rfm[rfm['RFM_SCORE'] == "555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12415.00000,12,7,19543.84000,5,5,5,555
12431.00000,10,13,4370.52000,5,5,5,555
12471.00000,11,49,20139.74000,5,5,5,555
12472.00000,6,13,11308.48000,5,5,5,555
12474.00000,15,13,5048.66000,5,5,5,555


## Naming the RFM Scores

the following dict has been made according to the famous RFM graphic

In [29]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

we will be using Recency and Frequency scores for customer segmentation. We are assuming that a customer who has recently purchased and who is often purchasing should have high RFM scores.

In [30]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,25
12347.00000,4,2,1323.32000,5,2,4,524,52
12348.00000,75,1,222.16000,2,1,1,211,21
12349.00000,44,3,2671.14000,3,3,5,335,33
12351.00000,12,1,300.93000,5,1,2,512,51


in the last step, we will convert the metrics into category names

In [31]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,2,5,2,252,Cant_Loose
12347.00000,4,2,1323.32000,5,2,4,524,Potential_Loyalists
12348.00000,75,1,222.16000,2,1,1,211,Hibernating
12349.00000,44,3,2671.14000,3,3,5,335,Need_Attention
12351.00000,12,1,300.93000,5,1,2,512,New_Customers


see the number of customers that fall into each category

In [32]:
rfm['Segment'].value_counts()

Hibernating            1017
Loyal_Customers         742
Champions               663
At_Risk                 611
Potential_Loyalists     517
About_to_Sleep          343
Need_Attention          207
Promising                87
Cant_Loose               77
New_Customers            50
Name: Segment, dtype: int64

## Time for Action

Now that we have all the scores for the customers and we have been able to categorize them into 10 groups, it's time for action. We will be using metrics for this process, not scores. We will be focusing on the groups that need a better customer relationship and try to figure out what we can do in order to make that specific segment purchase more frequently and become loyal. Thanks to RFM scores of the segments, we know what exactly that segment needs.

In [33]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

Recency       Frequency         Monetary      
                         mean count      mean count       mean count
Segment                                                             
About_to_Sleep       54.81924   343   1.20117   343  441.32000   343
At_Risk             153.15548   611   3.07365   611 1188.87832   611
Cant_Loose          125.11688    77   9.11688    77 4099.45000    77
Champions             8.11916   663  12.55354   663 6852.26417   663
Hibernating         215.05605  1017   1.12586  1017  403.18339  1017
Loyal_Customers      37.28706   742   6.83019   742 2746.06735   742
Need_Attention       54.26570   207   2.44928   207 1060.35700   207
New_Customers         9.58000    50   1.00000    50  386.19920    50
Potential_Loyalists  19.79304   517   2.01741   517  729.51099   517
Promising            26.74713    87   1.00000    87  367.08678    87

take a closer look at the customers that need attention:

In [34]:
rfm[rfm["Segment"] == "Need_Attention"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12349.00000,44,3,2671.14000,3,3,5,335,Need_Attention
12369.00000,50,3,1791.15000,3,3,4,334,Need_Attention
12371.00000,46,3,2179.42000,3,3,5,335,Need_Attention
12374.00000,58,3,2246.29000,3,3,5,335,Need_Attention
12389.00000,39,3,1433.33000,3,3,4,334,Need_Attention


**Need_Attention** segment has 207 customers that last purchased nearly 2 months ago, despite the fact that they don't frequently purchase, they spend quite a good amount of money. So we should be focusing on this group. In order to make them transform into a customer that purchases frequently, we can offer them some discount with a time limit of 30 days. So that they would revisit and purchase.

**Can't_Loose** segment has purchased for 9 times this year but the last date of this was nearly 4 months, they spend a good amount of money and they used to be our loyal customers, we can't loose them. We should put this 77 customers into our loyalty program, offer them seasonal discounts, make them feel special while purchasing from our company and make them loyal again. We can export the customer id list into an excel file and pass this file to our Marketing Department.  

In [35]:
marketing_df = pd.DataFrame()
marketing_df["Cant_Loose"] = rfm[rfm["Segment"] == "Cant_Loose"].index

In [36]:
marketing_df.head()

,Cant_Loose
0,12346.00000
1,12380.00000
2,12482.00000
3,12510.00000
4,12891.00000


change the dtype of Customer ID variable in order to get rid of the decimal part:

In [37]:
marketing_df['Cant_Loose'] = marketing_df['Cant_Loose'].astype(int)
marketing_df.head()

,Cant_Loose
0,12346
1,12380
2,12482
3,12510
4,12891


In [38]:
marketing_df.info

<bound method DataFrame.info of     Cant_Loose
0        12346
1        12380
2        12482
3        12510
4        12891
..         ...
72       18051
73       18064
74       18094
75       18251
76       18258

[77 rows x 1 columns]>

In [39]:
marketing_df.to_csv("cant_loose.csv")